In [0]:
# from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode
from pyspark.sql.types import StructType, StructField, StringType

# if __name__ == '__main__':
#     spark = (SparkSession.builder
#              .appName("Streaming App")
#              .master("local")
#              .config("spark.sql.shuffle.partitions",10)
#              .getOrCreate())

CRIME_SCHEMA = StructType(
    [
        StructField("code", StringType()),
        StructField("region", StringType()),
        StructField("category", StringType())
    ]
)
input_df = spark.readStream.load(format="CSV", path="/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/crime_data/input/", schema=CRIME_SCHEMA)

result_df = input_df.filter(col("region")=="Downtown")

result_df.writeStream.start(format="console", outputMode="APPEND").awaitTermination()

In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType
 
def ingest_data():
    CRIME_SCHEMA = StructType(
        [
            StructField("code", StringType()),
            StructField("region", StringType()),
            StructField("category", StringType()),
        ]
    )
    
    input_df = (
        spark.readStream.format("csv")
        .schema(CRIME_SCHEMA)
        .load(
            "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/crime_data/input/"
        )
    )
    
    result_df = input_df.filter(col("region") == "Downtown")
    
    result_df.writeStream.format("delta").outputMode("append").trigger(
        availableNow=True,
    ).option(
        "checkpointLocation",
        "/Volumes/quickstart_catalog/quickstart_schema/sandbox/dataset/crime_data/checkpoint",
    ).toTable(
        "quickstart_catalog.quickstart_schema.crime_tbl"
    )

ingest_data()

In [0]:
%sql

SELECT * from quickstart_catalog.quickstart_schema.crime_tbl;